<a href="https://colab.research.google.com/github/lalooedu/Algoritmos_de_Optimizaci-n/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos(V2_LUIS_AZOCAR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Luis Eduardo Azocar Arreaza  <br>
Url: https://github.com/lalooedu/Algoritmos_de_Optimizaci-n<br>
Google Colab: https://colab.research.google.com/drive/1ykGPXdOg5SIcZAihNbiklA_L5xQkn2-P#scrollTo=hVbXYX-RfPWh <br>
Problema:
>1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de una jornada de La Liga<br>
>3. Configuración de Tribunales

Descripción del problema:(copiar enunciado)


Problema 2. Organizar los horarios de partidos de La Liga(II)

• En primer lugar se clasifican los equipos en tres categorías según el numero de seguidores( que tiene relación directa con la audiencia). Hay 3 equipos en la
categoría A, 11 equipos de categoría B y 6 equipos de categoría C.

• Se conoce estadísticamente la audiencia que genera cada partido según los equipos que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos)

•  Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce según los coeficientes de la siguiente tabla

• Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes

• Es posible la coincidencia de horarios pero en este caso la audiencia de cada partido se verá afectada y se estima que se reduce en porcentaje según la
siguiente tabla dependiendo del número de coincidencias:

                                        

#Modelo
- ¿Como represento el espacio de soluciones?
El programa implementa un algoritmo de asignación heurística con evaluación de función objetivo, enfocado en maximizar la audiencia al asignar partidos de fútbol a distintos horarios.

Se basa en reglas simples y evaluaciones directas para encontrar una solución "buena" y no necesariamente óptima, maximizando una métrica coma la audiencia estimada, calculada como función del partido por equipos y el horario.

- ¿Cual es la función objetivo?

La función objetivo del algoritmo es maximizar la audiencia total televisiva de los partidos programados. Se calcula como:

audiencia_total = sum(base * ponderación)

- ¿Como implemento las restricciones?

Para la introduccion de los partidos obligatoriamente el lunes a las 20 y viernes a las 20, los parametros de los horarios deben ser introducidos de foma de entrada y se evalua si se ha asignado o no los partidos y en el caso de que haya sido a si, sale un mensaje de que seben asignarse obligatoriamente parteeidos en estos horarios.

ejemplo de la introducción: V20,S12,S16,S18,S20,D12,D18,D18,D20,L20

Para el caso de que si hay coincidencia de partidos en el mismo horario se evaluan las coincidencias y se vuelve a multiplicar la ponderacion de audiencia de acuerdo a su clasificacion.

  if repeticiones > 1:
      nueva_asignacion[i][6] = nueva_asignacion[i][5] * ponderacion_horarios[horario]

la funcion partido me hace asignar de forma aleatoria los equipos, de forma de asignar las mejores audiencias, pero podrian introducirce una lista de combinaciones de equipos y categoria en la funcion # Generar partidos

optimizando la asignación
partidos, categorias = generar_partidos(equipos)
mejor_asignacion, mejor_audiencia = optimizar_asignacion(partidos, categorias, horarios)

In [28]:
from itertools import permutations
import pandas as pd
import random
from collections import Counter

# Datos de equipos por categoría
equipos = {
    'A': ['Barcelona', 'Real Madrid', 'Atlético'],
    'B': ['Valencia', 'Sevilla', 'Athletic', 'Villarreal', 'Real Sociedad', 'Celta',
          'Espanyol', 'Betis', 'Granada', 'Levante', 'Getafe'],
    'C': ['Mallorca', 'Eibar', 'Osasuna', 'Leganés', 'Valladolid', 'Alavés']
}

# Posibles horarios de este modo hacemos que se introduzca obligatoriamente un horario para el lunes a las 20 y viernes a las 20
def ingresar_horarios():
    entrada = input("Ingrese 10 horarios separados por coma (ej: V20,S12,...): ")
    horariosinput = [h.strip() for h in entrada.split(',') if h.strip() != '']
    horarios=[]
    if 'V20' not in horariosinput or 'L20' not in horariosinput:
        print("Error: Debe incluir al menos un partido el viernes a las 20 y uno el lunes 20.")
        exit()
    else :
        horarios = horariosinput
    return horarios

horarios = ingresar_horarios()# Posibles horarios


#horarios = ['V20', 'S12', 'S16', 'S18', 'S20', 'D12', 'D18', 'D18', 'D20', 'L20']
#Ejemplo de introduccion de parametros
## V20,S12,S16,S18,S20,D12,D18,D18,D20,L20

# Audiencia base por categoría
audiencia_base = {
    ('A', 'A'): 2,
    ('A', 'B'): 1.3,
    ('A', 'C'): 1,
    ('B', 'B'): 0.9,
    ('B', 'C'): 0.75,
    ('C', 'C'): 0.47
}

# Ponderación por horario
ponderacion_horarios = {
    'V20': 0.4, 'S12': 0.55, 'S16': 0.7, 'S18': 0.8, 'S20': 1,
    'D12': 0.45, 'D16': 0.75, 'D18': 0.85, 'D20': 1, 'L20': 0.4
}

# Generar combinaciones de partidos sin repeticiones excesivas
def generar_partidos(equipos):
    partidos = []
    categorias = []
    equipos_disponibles = set()

    for categoria1 in equipos:
        for categoria2 in equipos:
            for equipo1 in equipos[categoria1]:
                for equipo2 in equipos[categoria2]:
                    if equipo1 != equipo2 and equipo1 not in equipos_disponibles and equipo2 not in equipos_disponibles:
                        partidos.append((equipo1, equipo2))
                        categorias.append((categoria1, categoria2))
                        equipos_disponibles.update([equipo1, equipo2])
                        if len(partidos) == 10:
                            return partidos, categorias
    return partidos, categorias

# Función objetivo
def calcular_audiencia(categoria, horario):
    base = audiencia_base.get(categoria, 0)
    ponderacion = ponderacion_horarios.get(horario, 1)
    base_ponderacion = base * ponderacion
    return base, ponderacion, base_ponderacion

# Buscar la mejor asignación de partidos
def optimizar_asignacion(partidos, categorias, horarios):
    mejor_audiencia = 0
    mejor_asignacion = []
    random.shuffle(horarios)

    coincidencias = Counter()
    nueva_asignacion = []

    for partido, categoria, horario in zip(partidos, categorias, horarios):
        coincidencias[horario] += 1
        base, ponderacion, base_ponderacion = calcular_audiencia(categoria, horario)
        nueva_asignacion.append([horario, partido, categoria, base, ponderacion, base_ponderacion, base_ponderacion])

    # Aplicar corrección a todas las filas con horarios duplicados
    for i in range(len(nueva_asignacion)):
        horario = nueva_asignacion[i][0]
        # Número de veces que aparece el horario
        repeticiones = coincidencias[horario]
         # Recalcular Corrección Coincidencia
        if repeticiones > 1:
            nueva_asignacion[i][6] = nueva_asignacion[i][5] * ponderacion_horarios[horario]

    audiencia_total = sum(aud for _, _, _, _, _, _, aud in nueva_asignacion)
    if audiencia_total > mejor_audiencia:
        mejor_audiencia = audiencia_total
        mejor_asignacion = nueva_asignacion

    return mejor_asignacion, mejor_audiencia

# Generar partidos optimizando la asignación
partidos, categorias = generar_partidos(equipos)
mejor_asignacion, mejor_audiencia = optimizar_asignacion(partidos, categorias, horarios)

# Tabla de resultados
tabla = pd.DataFrame(mejor_asignacion, columns=["Horario", "Partido", "Categoría", "Base", "Ponderación", "Base * Ponderación", "Corrección Coincidencia"])
display(tabla)
print(f"\nAudiencia máxima es de: {mejor_audiencia:.2f} millones")

Ingrese 10 horarios separados por coma (ej: V20,S12,...): V20,S12,S16,S18,S20,D12,D18,D18,D20,L20


,Horario,Partido,Categoría,Base,Ponderación,Base * Ponderación,Corrección Coincidencia
0,S12,"(Barcelona, Real Madrid)","(A, A)",2.00,0.55,1.1000,1.100000
1,S18,"(Atlético, Valencia)","(A, B)",1.30,0.80,1.0400,1.040000
2,L20,"(Sevilla, Athletic)","(B, B)",0.90,0.40,0.3600,0.360000
3,S16,"(Villarreal, Real Sociedad)","(B, B)",0.90,0.70,0.6300,0.630000
4,D12,"(Celta, Espanyol)","(B, B)",0.90,0.45,0.4050,0.405000
5,V20,"(Betis, Granada)","(B, B)",0.90,0.40,0.3600,0.360000
6,S20,"(Levante, Getafe)","(B, B)",0.90,1.00,0.9000,0.900000
7,D18,"(Mallorca, Eibar)","(C, C)",0.47,0.85,0.3995,0.339575
8,D20,"(Osasuna, Leganés)","(C, C)",0.47,1.00,0.4700,0.470000
9,D18,"(Valladolid, Alavés)","(C, C)",0.47,0.85,0.3995,0.339575



Audiencia máxima es de: 5.94 millones


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

In [ ]:
#Respuesta

O(n^2 + m + k)

siendo:

n el número total de equipos,
m el número de partidos a asignar,
k el número de horarios proporcionados.

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

#Respuesta

Se utilizo un algoritmo heurístico con una combinacion de algoritmos de generación y evaluación de soluciones. La Heurística de optimización, utiliza una combinación de pesos como ponderacion horarios y bases de audiencia para evaluar y optimizar las asignaciones de partidos y horarios. Este enfoque no garantiza la solución globalmente óptima, pero encuentra una solución razonable dentro del espacio de búsqueda, lo que es común en algoritmos heurísticos.

Asignación aleatoria elemento probabilístico:

La función random.shuffle(horarios) introduce aleatoriedad en la asignación de horarios, lo que implica que tu algoritmo explora diferentes combinaciones posibles.

División en subproblemas, el problema principal asignar partidos a horarios maximizando la audiencia se divide en subproblemas:

Generar combinaciones de partidos, evaluar la audiencia asignada en base a horarios y categorías (calcular_audiencia). Optimizar la asignación considerando restricciones y ponderaciones (optimizar_asignacion).

En resumen, tu programa pertenece a la familia de algoritmos heurísticos con enfoque de optimización por asignación y búsqueda iterativa, con características de corrección greedy y elementos aleatorios.
